<a href="https://colab.research.google.com/github/SudipBhakta/Vehicle_usage_forecast/blob/main/Rename_ANd_Attached_PO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# =========================================
# Google Colab: Excel + PDF → Rename → Add PO → ZIP (FIXED)
# =========================================

!pip install pandas openpyxl PyMuPDF

import pandas as pd
import os
import shutil
import tempfile
import zipfile
from datetime import datetime
from google.colab import files
import fitz  # PyMuPDF

# -------- Step 1: Upload Excel --------
print("📤 Upload Excel file:")
excel_upload = files.upload()
excel_file = list(excel_upload.keys())[0]

df = pd.read_excel(excel_file)

# Column mapping (0-based index)
file_col = df.columns[4]  # Column E (File Name)
po_col = df.columns[1]    # Column B (PO Number)

rename_values = df[file_col].dropna().astype(str).str.strip().tolist()
po_map = dict(zip(
    df[file_col].astype(str).str.strip(),
    df[po_col].astype(str).str.strip()
))

# -------- Step 2: Upload PDFs --------
print("\n📂 Upload ALL PDF files together:")
pdf_upload = files.upload()

# -------- Step 3: Create temp folders --------
temp_dir = tempfile.mkdtemp()
input_dir = os.path.join(temp_dir, "input_pdfs")
renamed_dir = os.path.join(temp_dir, "renamed_pdfs")
output_dir = os.path.join(temp_dir, "final_pdfs")

os.makedirs(input_dir)
os.makedirs(renamed_dir)
os.makedirs(output_dir)

# -------- Save PDFs --------
pdf_paths = []
for name, data in pdf_upload.items():
    if name.lower().endswith(".pdf"):
        path = os.path.join(input_dir, name)
        with open(path, "wb") as f:
            f.write(data)
        pdf_paths.append(path)

# -------- Sort PDFs by modified time --------
def get_pdf_time(path):
    return datetime.fromtimestamp(os.path.getmtime(path))

pdf_paths.sort(key=get_pdf_time)

# -------- Step 4: Rename PDFs --------
for i, pdf in enumerate(pdf_paths):
    if i >= len(rename_values):
        break
    new_name = rename_values[i] + ".pdf"
    shutil.copy(pdf, os.path.join(renamed_dir, new_name))

# -------- Step 5: Add PO Number --------
processed = 0

for pdf_file in os.listdir(renamed_dir):
    if not pdf_file.lower().endswith(".pdf"):
        continue

    base_name = os.path.splitext(pdf_file)[0]
    po_number = po_map.get(base_name)

    if not po_number:
        print(f"⚠️ PO not found for {pdf_file}")
        continue

    input_path = os.path.join(renamed_dir, pdf_file)
    output_path = os.path.join(output_dir, pdf_file)

    doc = fitz.open(input_path)

    for page in doc:  # add PO on ALL pages
        text = f"PO Number: {po_number}"
        font_size = 14
        text_width = fitz.get_text_length(text, fontsize=font_size)
        x = (page.rect.width - text_width) / 2
        y = 30

        page.insert_text(
            (x, y),
            text,
            fontsize=font_size,
            color=(0, 0, 1)
        )

    doc.save(output_path)
    doc.close()
    processed += 1

print(f"\n✅ {processed} PDFs processed")

# -------- Step 6: Create ZIP OUTSIDE temp dir --------
output_zip = "Final_PDFs.zip"

with zipfile.ZipFile(output_zip, "w", zipfile.ZIP_DEFLATED) as zipf:
    for file in os.listdir(output_dir):
        zipf.write(os.path.join(output_dir, file), arcname=file)

print("📦 ZIP ready for download")
files.download(output_zip)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 91.1 MB/s eta 0:00:00
📤 Upload Excel file:


TypeError: 'NoneType' object is not subscriptable